In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
n_collect = 200

In [3]:
np.linspace(0, 0.8, 33)

array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
       0.225, 0.25 , 0.275, 0.3  , 0.325, 0.35 , 0.375, 0.4  , 0.425,
       0.45 , 0.475, 0.5  , 0.525, 0.55 , 0.575, 0.6  , 0.625, 0.65 ,
       0.675, 0.7  , 0.725, 0.75 , 0.775, 0.8  ])

In [4]:
def collect_vul(pars_post, n_collect=20, coverages=np.linspace(0, 0.8, 33), tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, ms, _ = m.simulate_onward(y1, pars, intv={tp: {'Coverage': k}}, t_end=2031)

            ms = ms[idx].reset_index().assign(Key = i, Coverage=k, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Coverage': k, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum()
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Type=tp)
    mss_stats = pd.DataFrame(mss_stats).assign(Type=tp)
    return mss, mss_stats

In [5]:
for folder in ['dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)

    mss0, mss_stats0 = collect_vul(pars_post, n_collect=n_collect, tp='VulACF')
    mss1, mss_stats1 = collect_vul(pars_post, n_collect=n_collect, tp='PlainACF')
    mss = pd.concat([mss0, mss1])
    mss_stats = pd.concat([mss_stats0, mss_stats1])
    mss.to_csv(f'{out_path}/Sim_VulACF_budget.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_budget_stats.csv')

dy_lo
dy_hi


In [6]:
def collect_fu(pars_post, n_collect=20, coverage=0.03, tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    intvs = {
        'Baseline': {},
        'Vul_0_0': {'VulACF': {'Coverage': coverage, 'FollowUp': 0, 'Duration': 0}},
        'Vul_2_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 2}},
        'Vul_2_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 2}},
        'Vul_3_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 3}},
        'Vul_3_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 3}},
    }
    
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
        
        for key, intv in intvs.items():
            _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)

            ms = ms[idx].reset_index().assign(Key = i, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Scenario': key, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum()
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [7]:
for folder in ['dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fu(pars_post, n_collect=n_collect, coverage=0.03)
    mss.to_csv(f'{out_path}/Sim_VulACF_followup.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_followup_stats.csv')

dy_lo
dy_hi


In [8]:
mss_stats

,Key,Scenario,Pop0,IncR,MorR,ACF_Vul_Footfall,ACF_Vul_Screened,ACF_Vul_Confirmed,ACF_Vul_Yield,ACF_Vul_TP,...,ACF_Vul_DR_Sl,ACF_Plain_Footfall,ACF_Plain_Screened,ACF_Plain_Confirmed,ACF_Plain_Yield,ACF_Plain_TP,ACF_Plain_DS_Fl,ACF_Plain_DR_Fl,ACF_Plain_DR_Sl,Coverage
0,7,Baseline,2.382882,0.138768,0.020040,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1,7,Vul_0_0,2.382882,0.130996,0.018200,7.427207,1.322207,0.342026,0.020951,0.008374,...,0.000429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
2,7,Vul_2_3,2.382882,0.124355,0.016400,9.390678,1.322207,0.341984,0.020899,0.008321,...,0.000433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
3,7,Vul_2_6,2.382882,0.125900,0.016821,9.387364,1.322207,0.342134,0.021084,0.008508,...,0.000442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
4,7,Vul_3_3,2.382882,0.122808,0.015946,10.325083,1.322207,0.341981,0.020895,0.008317,...,0.000436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1490,Vul_0_0,2.383804,0.143141,0.016652,7.427280,1.322719,0.332451,0.014518,0.005600,...,0.000227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1196,1490,Vul_2_3,2.383804,0.136599,0.015267,9.459881,1.322719,0.332384,0.014437,0.005518,...,0.000226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1197,1490,Vul_2_6,2.383804,0.138137,0.015597,9.459755,1.322719,0.332481,0.014555,0.005637,...,0.000231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1198,1490,Vul_3_3,2.383804,0.135063,0.014917,10.447341,1.322719,0.332367,0.014417,0.005498,...,0.000226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03


In [9]:
def collect_fux(pars_post, n_collect=20, coverage=0.03, tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
    
        _, ms, _ = m.simulate_onward(y1, pars, intv={}, t_end=2031)

        ms = ms[idx].reset_index().assign(Key=i, FollowUp=0, Duration=0)
        mss.append(ms)
        ms_stats = {'Key': i, 'Scenario': 'Baseline', 'Pop0': ms.Pop.iloc[0], 
                    'FollowUp': 0, 'Duration': 0}
        for key in idx[1:]:
            ms_stats[key] = (ms[key] * ms.Pop).sum()

        mss_stats.append(ms_stats)
    
        for n_fu in list(range(1, 13)): # + [24, 30, 36]:
            fu = 1 / n_fu
            for dur in np.linspace(0, 4, 9):
                intv = {'VulACF': {'Coverage': coverage, 'FollowUp': fu, 'Duration': dur}}
                
                _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)
                try:
                    ms = ms[idx].reset_index().assign(Key = i, FollowUp=fu, Duration=dur)
                    mss.append(ms)

                    ms_stats = {'Key': i, 'Scenario': key, 'Pop0': ms.Pop.iloc[0], 
                                'FollowUp': fu, 'Duration': dur}

                    for key in idx[1:]:
                        ms_stats[key] = (ms[key] * ms.Pop).sum()

                    mss_stats.append(ms_stats)
                except TypeError:
                    pass
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [ ]:
for folder in ['dy_hi', 'dy_lo']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fux(pars_post, n_collect=200, coverage=0.03)
    mss.to_csv(f'{out_path}/Sim_VulACF_fudur.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_fudur_stats.csv')

dy_hi
dy_lo


/home/jovyan/work/sim/components/transmission.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  foi = (trans * y).sum() / y.sum()
/home/jovyan/work/sim/components/transmission.py:10: RuntimeWarning: invalid value encountered in multiply
  return (sus * y) * foi
/home/jovyan/work/sim/intv/proc_demo.py:25: RuntimeWarning: invalid value encountered in multiply
  calc['deaths_tb'] = dr_tb * y
/home/jovyan/work/sim/components/transmission.py:9: RuntimeWarning: invalid value encountered in multiply
  foi = (trans * y).sum() / y.sum()
/opt/conda/envs/pymc-dev/lib/python3.10/site-packages/numpy/core/_methods.py:48: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/jovyan/work/sim/intv/proc_acf.py:84: RuntimeWarning: invalid value encountered in multiply
  r_acf = n_target / (eli * y[:, :2]).sum()
/home/jovyan/work/sim/intv/proc_acf.py:87: RuntimeWarning: invalid value encountered in multiply
  screened =

In [ ]:
mss_stats